In [ ]:
%%capture
!pip install git+https://github.com/optuna/optuna.git #installa la libreria optuna

In [ ]:
import pandas as pd
import numpy as np
import math as mh

import seaborn as sns
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn import linear_model

from keras import *
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from tensorflow.keras import regularizers
from keras.optimizers import Adam, RMSprop
from keras.backend import clear_session

import optuna
from optuna.samplers import RandomSampler
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train = pd.read_csv("/content/drive/MyDrive/AML_project/train_manipulated_integrated.csv")

In [ ]:
y_train = train['trip_duration'].values.reshape(-1, 1)

X_train = train.drop(columns = ['trip_duration'])

#divisione train test
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.30,
                                                  random_state = 1234)

In [ ]:
#divisione train validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.15,
                                                  random_state = 1234)

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(857057, 48)
(151246, 48)
(432130, 48)


In [ ]:
#si separano le dummies dalle quantitative

X_train_to_stand = X_train[['passenger_count', 'distance',
                                            'maximum temperature', 'minimum temperature',
                                           'average temperature',
                                           'precipitation', 
                                           'snow fall',
                                           'snow depth', 'direction']]

X_train_not_stand = X_train.drop(columns = ['passenger_count', 'distance',
                                            'maximum temperature', 'minimum temperature',
                                           'average temperature',
                                           'precipitation', 
                                           'snow fall',
                                           'snow depth', 'direction']).values


X_val_to_stand = X_val[['passenger_count', 'distance',
                                            'maximum temperature', 'minimum temperature',
                                           'average temperature',
                                           'precipitation', 
                                           'snow fall',
                                           'snow depth', 'direction']]

X_val_not_stand = X_val.drop(columns = ['passenger_count', 'distance',
                                            'maximum temperature', 'minimum temperature',
                                           'average temperature',
                                           'precipitation', 
                                           'snow fall',
                                           'snow depth', 'direction']).values

X_test_to_stand = X_test[['passenger_count', 'distance',
                                            'maximum temperature', 'minimum temperature',
                                           'average temperature',
                                           'precipitation', 
                                           'snow fall',
                                           'snow depth', 'direction']]

X_test_not_stand = X_test.drop(columns = ['passenger_count', 'distance',
                                            'maximum temperature', 'minimum temperature',
                                           'average temperature',
                                           'precipitation', 
                                           'snow fall',
                                           'snow depth', 'direction']).values

In [ ]:
#standardizzazione

scaler = StandardScaler()

X_train_stand = scaler.fit_transform(X_train_to_stand)
y_train_stand = scaler.fit(y_train).transform(y_train)

X_val_stand = scaler.fit(X_train_to_stand).transform(X_val_to_stand)
y_val_stand = scaler.fit(y_train).transform(y_val)

X_test_stand = scaler.fit(X_train_to_stand).transform(X_test_to_stand)
y_test_stand = scaler.fit(y_train).transform(y_test)

In [ ]:
X_train = np.concatenate((X_train_stand, X_train_not_stand), axis = 1)
X_val = np.concatenate((X_val_stand, X_val_not_stand), axis = 1)
X_test = np.concatenate((X_test_stand, X_test_not_stand), axis = 1)

y_train = y_train_stand
y_val = y_val_stand
y_test = y_test_stand

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(857057, 48)
(151246, 48)
(432130, 48)


In [ ]:
### inizia l'ottimizzazione

def objective(trial): 
  clear_session()
  initializer = tf.keras.initializers.GlorotUniform(seed=1234) #inizializiamo i pesi

  #callback di early stopping
  es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', 
                                        patience=5, 
                                        restore_best_weights=True)
  
  callbacks_list = [optuna.integration.KerasPruningCallback(trial, 'val_loss'), 
                      es]

  #definizione dell'ottimizzatore
  opt = tf.keras.optimizers.SGD(learning_rate= 0.002, momentum=0.3, 
                              nesterov = False)
  #creazione dello spazio di ricerca

  n_hidden = trial.suggest_categorical("#hidden", [3, 4])

  if n_hidden == 3:
    units_1 = trial.suggest_int("dense1", 32, 256, step = 16)
    units_2 = trial.suggest_int("dense2", 32, 128, step = 16)
    units_3 = trial.suggest_int("dense3", 32, 128, step = 16)
    dropout = trial.suggest_categorical('dropout', [0.1, 0.2, 0.3, 0.4, 0.5])
    activations = 'relu'

    model = Sequential()
    model.add(Dense(units_1, input_shape = (48,),
                  activation = activations, 
                  kernel_initializer=initializer))
    model.add(Dropout(dropout, seed=1000))
    model.add(Dense(units_2,
                  activation = activations, 
                  kernel_initializer=initializer))
    model.add(Dropout(dropout, seed=1000))
    model.add(Dense(units_3,
                  activation = activations, 
                  kernel_initializer=initializer))
    model.add(Dropout(dropout, seed=1000))
    model.add(Dense(1,
                  activation = "linear", 
                  kernel_initializer=initializer))
  else:
    units_1 = trial.suggest_int("dense1", 32, 256, step = 16)
    units_2 = trial.suggest_int("dense2", 32, 128, step = 16)
    units_3 = trial.suggest_int("dense3", 32, 128, step = 16)
    units_4 = trial.suggest_int("dense4", 16, 64, step = 16)
    dropout = trial.suggest_categorical('dropout', [0.1, 0.2, 0.3, 0.4, 0.5])
    activations = 'relu'

    model = Sequential()
    model.add(Dense(units_1, input_shape = (48,),
                  activation = activations, 
                  kernel_initializer=initializer))
    model.add(Dropout(dropout, seed=1000))
    model.add(Dense(units_2,
                  activation = activations, 
                  kernel_initializer=initializer))
    model.add(Dropout(dropout, seed=1000))
    model.add(Dense(units_3,
                  activation = activations, 
                  kernel_initializer=initializer))
    model.add(Dropout(dropout, seed=1000))
    model.add(Dense(units_4,
                  activation = activations, 
                  kernel_initializer=initializer))
    model.add(Dropout(dropout, seed=1000))
    model.add(Dense(1,
                  activation = "linear", 
                  kernel_initializer=initializer))
    
  model.compile(loss = "mse", optimizer = opt, 
                metrics = [tf.keras.metrics.RootMeanSquaredError()])

  model.fit(X_train, y_train, epochs = 30, 
            batch_size = 128, validation_data=(X_val, y_val),
            shuffle = True,
            callbacks = callbacks_list,
            verbose=False)
  
  score = model.evaluate(X_val, y_val, verbose=0, batch_size = 128)

  return score[0] #si minimizza il MSE 


In [ ]:
sampler = RandomSampler(10) #random seaerch

study = optuna.create_study(study_name = "DROPOUT_", direction="minimize", 
                            sampler = sampler, storage='sqlite:////content/drive/MyDrive/AML_project/sqlite/DROPOUT_.db', 
                            load_if_exists=True)

[I 2021-01-15 08:34:38,978] Using an existing study with name 'DROPOUT_' instead of creating a new one.


In [ ]:
%%time
%%capture

study.optimize(objective, n_trials=50)

[I 2021-01-13 14:06:17,349] Trial 0 finished with value: 0.5612729787826538 and parameters: {'#hidden': 4, 'dense1': 240, 'dense2': 96, 'dense3': 32, 'dense4': 32, 'dropout': 0.4}. Best is trial 0 with value: 0.5612729787826538.
[I 2021-01-13 14:12:33,164] Trial 1 finished with value: 0.2663886249065399 and parameters: {'#hidden': 3, 'dense1': 176, 'dense2': 112, 'dense3': 32, 'dropout': 0.2}. Best is trial 1 with value: 0.2663886249065399.
[I 2021-01-13 14:17:57,698] Trial 2 finished with value: 0.2771306037902832 and parameters: {'#hidden': 3, 'dense1': 160, 'dense2': 48, 'dense3': 32, 'dropout': 0.3}. Best is trial 1 with value: 0.2663886249065399.
[I 2021-01-13 14:24:13,266] Trial 3 finished with value: 0.26256826519966125 and parameters: {'#hidden': 3, 'dense1': 128, 'dense2': 96, 'dense3': 80, 'dropout': 0.1}. Best is trial 3 with value: 0.26256826519966125.
[I 2021-01-13 14:30:25,147] Trial 4 finished with value: 0.27288228273391724 and parameters: {'#hidden': 3, 'dense1': 208, 

CPU times: user 1h 27min 9s, sys: 7min 29s, total: 1h 34min 38s
Wall time: 1h 6min 49s


In [ ]:
print('Best trial number: ', study.best_trial.number)

print("Number of finished trials: {}".format(len(study.trials)))
trial = study.best_trial

print("  Value: {}".format(trial.value))
print('Best parameters: \n', study.best_trial.params)

Best trial number:  9
Number of finished trials: 50
  Value: 0.2565009295940399
Best parameters: 
 {'#hidden': 3, 'dense1': 240, 'dense2': 112, 'dense3': 128, 'dropout': 0.1}


In [ ]:
df_trials = study.trials_dataframe()

In [ ]:
#si salvano i risultati dei tenativi in un csv
df_trials.to_csv("/content/drive/MyDrive/AML_project/modelli/modelli salvati/opt_path/dropout.csv", index = False)